# OUTLINE
 1 Python basics : Setup, syntax & standard libraries
        
 2 Pymatgen 1 : build a structure & write a vasp job
    
__3 Python + slurm + bash : launch a vasp job__
    
 4 Pymatgen 2 : read a vasp job & plot DOS
    
 5 Matplotlib : make your own plots 
    
 6 Pymatgen 3 : advanced functionalities (disorder, bader, lobster ... )
    
 7 the read_write library : installation, work flow & examples


# Session 3 : Python + slurm + bash : launch a vasp job
* ssh / sshfs 
* slurm syntax (sbatch, scontrol)
* python subprocess module

## Setting ssh link to server

### within the ICGM 
direct access

In [ ]:
# ~/.ssh/config file
Host direct_frodon
	HostName frodon.lsd.univ-montp2.fr
	User jvergnet
    # add rsa key if desired for swift login

on the console 

-X to allow the display of window via ssh

    ssh direct_frodon -X

### Outside of the ICGM 
access through getaway redicrection

#### Getaway (loco2)
Configure access & forwarding from local port to frodon port

(Add control master socket for ssh multiplexing )

rsa key is useless here since loco2 will still ask you for ubikey

In [ ]:
# ~/.ssh/config file
Host loco2
	HostName loco2.lsd.univ-montp2.fr
	User jvergnet

    # loco only listen to port 2222
    Port 2222

    # syntax LocalForward $local_port $address_of_end_machine:$port_on_machine 
    # port_on_machine is 22 for ssh (by default)
	LocalForward 2221 frodon.lsd.univ-montp2.fr:22
	# LocalForward 2223 bipbip.lsd.univ-montp2.fr:22
	# LocalForward 2224 tornado.icgm.fr:22

    # SSH Multiplexing
    # monitor global connection when accessing several machines at once
	ControlMaster auto
	ControlPath ~/.ssh/master-socket/%r@%h:%p

    # Abort if there is a problem
    # usefull when running ssh headless (ssh loco -N) see below
    ExitOnForwardFailure yes

on the console (only once)

* -N tell that no ssh commands need to be executed (just forwarding)
* -f places your ssh process in the background 
after command execution or just after port forwarding if no command passed (-N))

    ssh -N -f loco2

#### Final host (e.g. frodon)
Access through local port forwarded via the getaway (loco2

In [ ]:
# ~/.ssh/config file
Host frodon_local
    User jvergnet
    
    # we send an ssh request on the LOCAL port 
	HostName localHost     

    # local_port number was defined in the Host loco2 description
    Port 2221
    
    # here you can add an rsa key on YOUR computer, paired with the final host 
    # see man ssh keygen
	IdentityFile /home/jvergnet/.ssh/k_rsa_fro_2
	

on the console (for every new window)

also works for bipbip or tornado if configured

-X to allow the display of window via ssh

    ssh frodon_local -X

### sshfs
Allow to access remote folder (on frodon machine) into your personal machine 

you "mount" the remote folder on an empty local folder, like a hard drive

    sshfs ssh_address:/home/user/remote_folder /home/user/empty_local_folder

eg, here : 

    sshfs frodon_local:/home/jvergnet/frofro /home/jvergnet/frofro

*I named them consistently on local and remote :
when developping scripts locally they can be used on the cluster without changing their execution path*

## Using Aliases 
Complexe commands can be assigned to simple expression which can be directly called from the console

The definition of such functions is done in the bash configuration file `~/.bash_aliases` (or `~/.bashrc` )

to use newly defined functions, we need to refresh this file by using `source ~/.bash_aliases`

*NB : See how the ssh calls are much more readable if `~/.ssh/config` is correctly set*

In [ ]:
# ~/.bash_aliases (or ~/.bashrc)

# initial command to setup a remote connection 
SSSH_OPEN_TUNNEL() {
	ssh -N -f loco2 &&  # open the getaway forwarding
	SSHFS_FRO &&  SSHFS_TOR && # mount both frodon and tornado on local folders (defined below)
	SSSH_TABS_OPEN # open a tab for each cluster (defined below)
	}

# initial command to close a remote connection 
# by destroying the master socket, all connection will be properly closed
SSSH_CLOSE_TUNNEL() {
	ssh -S ~/.ssh/master-socket/jvergnet@loco2.lsd.univ-montp2.fr\:2222 -O exit loco2
	}

# opening distinct ssh tabs in the console
SSSH_TABS_OPEN(){
	gnome-terminal --tab -- ssh frodon_local -X && 	gnome-terminal --tab -- ssh tornado_local -X
	}

# SSHFS commands
SSHFS_FRO() {
	sshfs frodon_local:/home/jvergnet/frofro /home/jvergnet/frofro
	}

SSHFS_FRO_DIRECT() {
        sshfs frodon:/home/jvergnet/frofro /home/jvergnet/frofro
        }

on the console (at the begining of your day)

to get one console tab on each cluster and a mounted filesystem for both

    SSSH_OPEN_TUNNEL

and if you want more terminal windows 

    SSSH_TABS_OPEN()

*Conclusion : work hard once, take it easy afterwads !*

## SLURM and job management

### Launching a job on a cluster (sbatch)

    sbatch vasp_job.sh

Whan a job is executed, there are 2 level of personalisation : 

1 the call to sbatch (slurm syntax) determines : 

* the name of the job `-J {}`
* the quality of service (QOS) `--qos=bg`
* the number of nodes per job `-N {} `
* the directory of execution `--workdir {}`
* *for arrays* : nb jobs simultaneaous and in total `--array=1-{tot}%{sim}`

2 the script which is executed by sbatch on each node determines : 

* All sbatch variables that were not determined during sbatch call (via `##NAME = VALUE` )
* file manipulation (e.g. `cp CONTCAR POSCAR` for vasp), 
* the type of executable (e.g. `vasp gamma`, `non-colinear` or `Gaussian` or else)
* *for arrays* : the diffeence between jobs of the same array

*nb : absolutely non-exhaustive list !*

then the run can be highly personalized : e.g. 

    sbatch -J LiCoO2  --qos=bg  -N 4 --workdir /path/to/jobfolder  vasp_job_non_colinear.sh

### Monitoring running Jobs 

#### Simple checking

`sinfo`  : number of occupied and idle nodes on the cluster

`squeue` : jobID, name, age and nb of nodes of the running jobs 

#### Advanced checking
Built-in control for queued or jobs

    scontrol show job $numjob

which returns (among other lines) : 

    StdOut=/path/to/file.stdout

this file collects the Standard output of the script called by sbatch.

We can include a reference to the working directory in the standard output to the job script : 

    echo 'echo \"JOBDIR=\"$PWD' >> vasp_job.sh

We can then acess the working directory and monitor the ongoing job : e.g. 

    tail $JOBDIR/OSZICAR 

*this works for all configuration, even when `--workdir` was specified in the command line*

This is summarized in the following bash script : 

In [ ]:
#!/bin/bash
# check_job 

# ============================
# GETTING the job folder name 

# get the line StdOut=/path/to/file.stdout
stdout_line=$(scontrol show job $numjob | grep StdOut)

# rip the "StdOut=" at the begining
stdout_file=${stdout_line#*=}

# get the line JOBDIR=/path/to/job_dir
workdir=$(cat $stdout_file |grep JOBDIR)

# rip the "JOBDIR=" at the begining
workdir=${workdir#*=}

cd $workdir

printf "\n ${workdir#*jvergnet/}\n"

# ============================
# ANALYSIS of running job folder

if grep -q F OSZICAR; #if "F" is present in the file 
then
        # echo " at least 1 ion movement"  
        # grep : 1 line of context -Before the -expression "F" in the -File OSZ
        # print all ionic steps and their last electronic step
        grep -B1 -e F -F OSZICAR | tail -n 10 

else
        # echo "not even 1 ion movement"
        # print the last 10 electronic steps
        tail -n 10 OSZICAR
fi


We can even automatize the extraction of jobID for the current user 

In [ ]:
#!/bin/bash
# check_job_all

# pipe results of "squeue" (line containing only jobID) to "while read" (line by line) 
squeue -h -u jvergnet -o %A | while read -r numjob rest_of_line 
do
        printf  "\n \n   ===> JOB No : %s " "$numjob"
        if [[ "$numjob" == *%* ]]; then # this lists the pending job of the job array
                echo " ==> Proxy job !"
        else :
                sh check_job $numjob # its a real running job 
        fi
done


## Launching job from python
The module `subprocess` allows a python script to issue commands to the terminal. 

In [ ]:
import subprocess

# allows to launch jobs on the cluster via python
subprocess.call(['sbatch vasp_job.sh'], shell=True)


Therefore the personalization can be done using a user friendly script and then properly formatted into a sbatch command string issued to the terminal. 

This has been done in the module `launch_job` that can be inspected in the file __launch_job.py__

In [ ]:
from launch_job import SlurmJob

project_folder = "/home/jvergnet/frofro/job_dir/"

slurm_job = SlurmJob(project_folder)
slurm_job.launch_job()

In [ ]:
to use this module, you need to set the vasp_job scripts to the proper location